In [1]:
pip install flask_ngrok

In [2]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np
import cv2

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
#from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok
from flask import send_file

from matplotlib import pyplot as plt

In [14]:
cd '/content/drive/MyDrive/yolo_deploy'

/content/drive/MyDrive/yolo_deploy


In [15]:
!mkdir /content/uploads

mkdir: cannot create directory ‘/content/uploads’: File exists


In [5]:

net = cv2.dnn.readNet('/content/drive/MyDrive/yolov3/yolov3_training_last.weights', '/content/drive/MyDrive/yolov3/yolov3_testing.cfg')

classes = []
with open("/content/drive/MyDrive/yolov3/classes.txt", "r") as f:
    classes = f.read().splitlines()

#cap = cv2.VideoCapture('video4.mp4')
#cap = 'test_images/<your_test_image>.jpg'
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

In [6]:
cv2.imread('/static/detections/result.png')

In [ ]:

# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/uploader', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = '/content/drive/MyDrive/yolo_deploy/static/uploads'
        file_path = os.path.join(
            basepath, secure_filename(f.filename))
        f.save(file_path)
        print('filename is :',secure_filename(f.filename))

        

        #while True:
        #_, img = cap.read()
        img = cv2.imread(file_path)
        print('read successful')
        height, width, _ = img.shape

        blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layerOutputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []
        print('next1')

        for output in layerOutputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.2:
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    x = int(center_x - w/2)
                    y = int(center_y - h/2)

                    boxes.append([x, y, w, h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)
        print('next2')

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)
        print('next3')

        if len(indexes)>0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i],2))
                color = colors[i]
                cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                cv2.putText(img, label + " " + confidence, (x, y+20), font, 1, (255,255,255), 2)
        print('next3')
        #cv2.imwrite('/content/drive/MyDrive/yolo_deploy/static/detections/result.png',img)
        cv2.imwrite(os.path.join('/content/drive/MyDrive/yolo_deploy/static/detections', secure_filename(f.filename)),img)
        #from google.colab.patches import cv2_imshow
        #cv2_imshow(img)
        #cv2.imshow('Image', img)
        plt.imshow(img)
        #result = plt.savefig('/content/drive/MyDrive/yolo_deploy/static/detections/result.png')
        #file_path_result = '/content/drive/MyDrive/yolo_deploy/static/detections/result.png'
        filename = secure_filename(f.filename)
        #filename = 'result.png'
        file_img = secure_filename(f.filename)
        print('next4')

        return render_template("uploaded.html", display_detection = filename, fname = file_img)
        print('done')
          #return send_file(('/content/result.jpg'), mimetype="image/jpeg", attachment_filename="result.jpg")
    return None


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b4b420c500d7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/May/2021 12:00:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 12:00:12] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 12:00:13] "GET /favicon.ico HTTP/1.1" 404 -


filename is : Cars89.png
read successful


...

127.0.0.1 - - [31/May/2021 12:00:30] "GET /static/uploads/Cars89.png HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 12:00:30] "GET /static/detections/Cars89.png HTTP/1.1" 200 -


filename is : Cars103.png
read successful


127.0.0.1 - - [31/May/2021 12:01:06] "POST /uploader HTTP/1.1" 200 -


next1
next2
next3
next3
next4


127.0.0.1 - - [31/May/2021 12:01:06] "GET /static/uploads/Cars103.png HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 12:01:06] "GET /static/detections/Cars103.png HTTP/1.1" 200 -
